    JACOB KNOPPING
    1/16/2020

### ASSIGNMENT 18.5.1. Weather model

For this assignment, you'll revisit the historical temperature dataset. To complete this assignment, submit a link a Jupyter notebook containing your solutions to the following tasks:

* First, load the dataset from the **weatherinszeged** table from Thinkful's database.
* Like in the previous checkpoint, build a linear regression model where your target variable is the difference between the *apparenttemperature* and the *temperature*. As explanatory variables, use *humidity* and *windspeed*. Now, estimate your model using OLS. What are the R-squared and adjusted R-squared values? Do you think they are satisfactory? Why? 
* Next, include the interaction of *humidity* and *windspeed* to the model above and estimate the model using OLS. Now, what is the R-squared of this model? Does this model improve upon the previous one? 
* Add *visibility* as an additional explanatory variable to the first model and estimate it. Did R-squared increase? What about adjusted R-squared? Compare the differences put on the table by the interaction term and the *visibility* in terms of the improvement in the adjusted R-squared. Which one is more useful?
* Choose the best one from the three models above with respect to their AIC and BIC scores. Validate your choice by discussing your justification with your mentor.

### SOLUTION

In [1]:
# import the relevant libraries:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import linear_model
import matplotlib.pyplot as plt
from sqlalchemy import create_engine

import warnings
warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'weatherinszeged'

In [2]:
#load the data from the PostgreSQL database
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

weather_df = pd.read_sql_query('SELECT * FROM weatherinszeged', con=engine)

#no need for an open connection (just the one query)
engine.dispose()

#view first 3 rows
weather_df.head(3)

,date,summary,preciptype,temperature,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure,dailysummary
0,2006-03-31 22:00:00+00:00,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.
1,2006-03-31 23:00:00+00:00,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.
2,2006-04-01 00:00:00+00:00,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.


In [3]:
#MODEL 1
#define target and explanatory variables
Y = weather_df['apparenttemperature'] - weather_df['temperature'] 
X = weather_df[['humidity', 'windspeed']]

#import stats models library
import statsmodels.api as sm

#add a constant
X = sm.add_constant(X)

#fit model
results = sm.OLS(Y, X).fit()

#get results summary
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.288
Model:                            OLS   Adj. R-squared:                  0.288
Method:                 Least Squares   F-statistic:                 1.949e+04
Date:                Thu, 16 Jan 2020   Prob (F-statistic):               0.00
Time:                        18:11:42   Log-Likelihood:            -1.7046e+05
No. Observations:               96453   AIC:                         3.409e+05
Df Residuals:                   96450   BIC:                         3.409e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.4381      0.021    115.948      0.000       2.397       2.479
humidity      -3.0292      0.024   -126.479      0.000      -3.076      -2.982
windspeed     -0.1193      0.001   -176.164      0.000      -0.121      -0.118
==============================================================================
Omnibus:                     3935.747   Durbin-Watson:                   0.267
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4613.311
Skew:                          -0.478   Prob(JB):                         0.00
Kurtosis:                       3.484   Cond. No.                         88.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The R-squared and adjusted R-squared values are both 0.288, which are very low. Or in other words, only 28.8% of the variance in the outcome is explained by the features/explanatory variables. In essence, the explanatory variables in the model are not explaining much about the outcome, or target variable. This is not a satisfactory model as this, for that reason.

In [4]:
#MODEL 2

#add a new variable for the interaction of humidity and windspeed
weather_df['humid_wind_inter'] = weather_df.humidity * weather_df.windspeed

#define target and explanatory variables
Y = weather_df['apparenttemperature'] - weather_df['temperature']
X = weather_df[['humidity', 'windspeed', 'humid_wind_inter']]

#import stats models library
import statsmodels.api as sm

#add a constant
X = sm.add_constant(X)

#fit model
results = sm.OLS(Y, X).fit()

#get results summary
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.341
Method:                 Least Squares   F-statistic:                 1.666e+04
Date:                Thu, 16 Jan 2020   Prob (F-statistic):               0.00
Time:                        18:11:42   Log-Likelihood:            -1.6669e+05
No. Observations:               96453   AIC:                         3.334e+05
Df Residuals:                   96449   BIC:                         3.334e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                0.0839      0.033      2.511      0.012       0.018       0.149
humidity             0.1775      0.043      4.133      0.000       0.093       0.262
windspeed            0.0905      0.002     36.797      0.000       0.086       0.095
humid_wind_inter    -0.2971      0.003    -88.470      0.000      -0.304      -0.291
==============================================================================
Omnibus:                     4849.937   Durbin-Watson:                   0.265
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9295.404
Skew:                          -0.378   Prob(JB):                         0.00
Kurtosis:                       4.320   Cond. No.                         193.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Adding the interaction of windspeed and humidity improved the R-squared values (0.341), however, not by enough to make the model satisfactory. The majority of variance in the model remains unexplained (65.9%).

In [5]:
#MODEL 3

#define target and explanatory variables

#ADDING VISIBILITY to the first model:
Y = weather_df['apparenttemperature'] - weather_df['temperature'] 
X = weather_df[['humidity', 'windspeed', 'visibility']]

#import stats models library
import statsmodels.api as sm

#add a constant
X = sm.add_constant(X)

#fit model
results = sm.OLS(Y, X).fit()

#get results summary
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.304
Model:                            OLS   Adj. R-squared:                  0.303
Method:                 Least Squares   F-statistic:                 1.401e+04
Date:                Thu, 16 Jan 2020   Prob (F-statistic):               0.00
Time:                        18:11:42   Log-Likelihood:            -1.6938e+05
No. Observations:               96453   AIC:                         3.388e+05
Df Residuals:                   96449   BIC:                         3.388e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.5756      0.028     56.605      0.000       1.521       1.630
humidity      -2.6066      0.025   -102.784      0.000      -2.656      -2.557
windspeed     -0.1199      0.001   -179.014      0.000      -0.121      -0.119
visibility     0.0540      0.001     46.614      0.000       0.052       0.056
==============================================================================
Omnibus:                     3833.895   Durbin-Watson:                   0.282
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4584.022
Skew:                          -0.459   Prob(JB):                         0.00
Kurtosis:                       3.545   Cond. No.                         131.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In model 3, visibility was added to what was model 1. This change marginally improved the adjusted R-squared value from 0.288 to 0.303. Again, this model doesn't explain much of the variance in the target, even less so than model 2.

_EXTRA:_

In [6]:
#MODEL 4
weather_df = pd.concat([weather_df,pd.get_dummies(weather_df.preciptype, prefix="preciptype", drop_first=True)], axis=1)
dummy_columns = list(pd.get_dummies(weather_df.preciptype, prefix="preciptype", drop_first=True).columns)

#add a new variable for the interaction of humidity and windspeed
weather_df['humid_wind_inter'] = weather_df.humidity * weather_df.windspeed

#define target and explanatory variables
Y = weather_df['apparenttemperature'] - weather_df['temperature']
X = weather_df[['humidity', 'windspeed', 'humid_wind_inter'] + dummy_columns]

#import stats models library
import statsmodels.api as sm

#add a constant
X = sm.add_constant(X)

#fit model
results = sm.OLS(Y, X).fit()

#get results summary
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.478
Model:                            OLS   Adj. R-squared:                  0.478
Method:                 Least Squares   F-statistic:                 1.770e+04
Date:                Thu, 16 Jan 2020   Prob (F-statistic):               0.00
Time:                        18:11:42   Log-Likelihood:            -1.5543e+05
No. Observations:               96453   AIC:                         3.109e+05
Df Residuals:                   96447   BIC:                         3.109e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -0.2646      0.061     -4.327      0.000      -0.385      -0.145
humidity             1.1109      0.039     28.735      0.000       1.035       1.187
windspeed            0.1011      0.002     46.202      0.000       0.097       0.105
humid_wind_inter    -0.3144      0.003   -105.138      0.000      -0.320      -0.309
preciptype_rain     -0.0950      0.053     -1.776      0.076      -0.200       0.010
preciptype_snow     -2.1304      0.055    -38.981      0.000      -2.238      -2.023
==============================================================================
Omnibus:                     1418.069   Durbin-Watson:                   0.379
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2657.755
Skew:                           0.057   Prob(JB):                         0.00
Kurtosis:                       3.805   Cond. No.                         380.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Out of models 1-3, model 2 has the highest adjusted R-sqaured value (0.341), and the lowest AIC and BIC scores (3.109e+05). Given that AIC and BIC take into consideration the sum of the squared errors (SSE), the sample size, and the number of parameters, the model with the lowest score is the most ideal by this metric. Therefore, model 2 is the best choice.

If you include my model 4, which takes into account precipitation, this would be the best choice out of all four (higher Adj. R-sqaured value of 0.478, lower AIC and BIC scores). In addition, the p value of the F statistic is 0.00. This tells us that the features in the model are useful at explaining the variance, in comparison to a reduced model.

In some cases such as weather, target variables are difficult to explain and even the best models only explain a very low portion of the variance in the target variable. In cases like this, satisfactory results is a matter of relative success. Therefore, model 4 can be considered a useful model.